In [1]:
import pandas as pd
#import plotly.graph_objects as go
# 출력 제한 해제
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_seq_items', None)
pd.options.display.max_colwidth = 100

import warnings
warnings.filterwarnings(action='ignore') 


In [2]:
train = pd.read_csv('./data/train.csv')
test =  pd.read_csv('./data/test.csv')
trade = pd.read_csv('./data/international_trade.csv')

train.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [3]:
time = pd.to_datetime(train['timestamp'].copy())

train['Date'] = pd.to_datetime(train['timestamp'])
train['week'] = train['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
train['day_name'] = train['Date'].dt.day_name()

train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))




In [4]:
import datetime
import holidays

# 공휴일 정보를 가져옴
holiday_list = holidays.Korea(years=datetime.datetime.now().year)

# train['holiday'] = 0

# # 특정 날짜가 공휴일인지 확인, 일요일 행은 train할때 다 지운다.
# for i in range(len(train)):
#     date = datetime.date(train.iloc[i,-4], train.iloc[i,-3], train.iloc[i,-2])
#     if date in holiday_list:
#         train.iloc[i,-1] = holiday_list[datetime.date(train.iloc[i, -4], train.iloc[i, -3], train.iloc[i, -2])] #공휴일이면 공휴일 이름 넣음

In [5]:
train.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),Date,week,day_name,year,month,day
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019-01-01,1,Tuesday,2019,1,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019-01-02,1,Wednesday,2019,1,2
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019-01-03,1,Thursday,2019,1,3
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019-01-04,1,Friday,2019,1,4
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019-01-05,1,Saturday,2019,1,5


In [6]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
test_re = test.copy()

test_re['year'] = test_re['timestamp'].apply(lambda x : int(x[0:4]))
test_re['month'] = test_re['timestamp'].apply(lambda x : int(x[5:7]))
test_re['day'] = test_re['timestamp'].apply(lambda x : int(x[8:10]))
test_re['Date'] = pd.to_datetime(test_re['timestamp'])
test_re['week'] = test_re['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
test_re['day_name'] = test_re['Date'].dt.day_name()


# test_re['holiday'] = 0

# for i in range(len(test_re)):
#     date = datetime.date(test_re.iloc[i,5], test_re.iloc[i,6], test_re.iloc[i,7])
#     if date in holiday_list:
#         test_re.iloc[i,-1] = holiday_list[datetime.date(test_re.iloc[i, 5], test_re.iloc[i, 6], test_re.iloc[i, 7])] #공휴일이면 공휴일 이름 넣음

In [7]:
test_re.head()

,ID,timestamp,item,corporation,location,year,month,day,Date,week,day_name
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,2023-03-04,9,Saturday
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,2023-03-05,9,Sunday
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,2023-03-06,10,Monday
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,2023-03-07,10,Tuesday
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,2023-03-08,10,Wednesday


In [21]:
from sklearn.preprocessing import LabelEncoder

# 793점 방법
train_x_regression = train[train['day_name'] != 'Sunday'].copy()

train_x_regression = train_x_regression.drop(columns=['ID', 'timestamp', 'supply(kg)','Date'])

train_x_classification = train.drop(columns=['ID', 'timestamp', 'supply(kg)','Date'])

for k in range(len(train_x_classification)):
    if train_x_classification['price(원/kg)'][k] > 0:
        train_x_classification['price(원/kg)'][k] = 1

test_x = test_re.drop(columns=['ID', 'timestamp','Date'])


qual_col = ['item', 'corporation', 'location','day_name']
#del train_x['holiday']

#train_x = pd.get_dummies(train_x, columns=qual_col, prefix=qual_col)
#test_x = pd.get_dummies(test_x, columns=qual_col, prefix=qual_col)

for i in qual_col:
   le = LabelEncoder()
   train_x_classification[i]=le.fit_transform(train_x_classification[i])
   train_x_regression[i]=le.transform(train_x_regression[i]) # reg는 sunday지웠으니..

   test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다
    

# %%
#train_x_regression = pd.concat([train_x_regression, train_y], axis=1)
#train_x_classification = pd.concat([train_x_classification, train_y], axis =1)

In [9]:
from pycaret.classification import *

c = setup(train_x_classification, 
          target = 'price(원/kg)',
          fold = 10, 
          #numeric_features=train_x_classification.columns, 
          session_id = 123, 
          normalize = True, 
          normalize_method = 'minmax', 
          train_size=0.8)


,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Binary
3,Original data shape,"(59397, 9)"
4,Transformed data shape,"(59397, 24)"
5,Transformed train set shape,"(47517, 24)"
6,Transformed test set shape,"(11880, 24)"
7,Ordinal features,1
8,Numeric features,4
9,Categorical features,4


In [10]:
m = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9424,0.9870,0.9502,0.9108,0.9301,0.8812,0.8818,2.1740
xgboost,Extreme Gradient Boosting,0.9416,0.9870,0.9484,0.9106,0.9291,0.8795,0.8801,0.9900
lightgbm,Light Gradient Boosting Machine,0.9367,0.9848,0.9527,0.8967,0.9238,0.8697,0.8709,0.2090
rf,Random Forest Classifier,0.9281,0.9810,0.9322,0.8941,0.9127,0.8517,0.8523,0.4280
dt,Decision Tree Classifier,0.9218,0.9185,0.9016,0.9042,0.9028,0.8374,0.8375,0.0750
et,Extra Trees Classifier,0.9204,0.9759,0.9186,0.8879,0.9029,0.8355,0.8359,0.6040
gbc,Gradient Boosting Classifier,0.9067,0.9602,0.9062,0.8681,0.8867,0.8074,0.8080,0.7410
knn,K Neighbors Classifier,0.9066,0.9536,0.8922,0.8780,0.8850,0.8064,0.8065,0.3500
ada,Ada Boost Classifier,0.8535,0.9320,0.7770,0.8470,0.8105,0.6915,0.6933,0.2610
svm,SVM - Linear Kernel,0.8489,0.0000,0.7550,0.8535,0.8008,0.6799,0.6836,0.0830


In [11]:
llar  = create_model('xgboost')
predictions_classfication = predict_model(llar, data = train_x_classification)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9377,0.9860,0.9541,0.8978,0.9251,0.8719,0.8730
1,0.9409,0.9867,0.9410,0.9148,0.9277,0.8777,0.8779
2,0.9402,0.9866,0.9494,0.9068,0.9276,0.8768,0.8774
3,0.9470,0.9886,0.9525,0.9189,0.9354,0.8904,0.8909
4,0.9415,0.9878,0.9509,0.9082,0.9291,0.8793,0.8800
5,0.9392,0.9869,0.9462,0.9069,0.9261,0.8745,0.8751
6,0.9396,0.9862,0.9483,0.9062,0.9268,0.8754,0.8761
7,0.9396,0.9868,0.9431,0.9103,0.9264,0.8752,0.8756
8,0.9472,0.9877,0.9567,0.9160,0.9359,0.8910,0.8916


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.9540,0.9918,0.9636,0.9253,0.9440,0.9050,0.9055


In [12]:
# 분류 결과 추가

In [23]:
train_df = pd.read_csv('./data/train.csv')[['ID','timestamp','supply(kg)', 'price(원/kg)']]
train_df['item_id'] = train_df.ID.str[0:6]

predictions_classfication_train_result = predict_model(llar, data = train_x_classification)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/shimjunseok/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py: │
│ 3526 in run_code                                                                                 │
│                                                                                                  │
│   3523 │   │   │   │   if async_:                                                                │
│   3524 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3525 │   │   │   │   else:                                                                     │
│ ❱ 3526 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3527 │   │   │   finally:                                                                      │
│   3528 │   │   │   │   # Reset our crash handler in place                                        │
│   3529 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 train_df = pd.read_csv('./data/train.csv')[['ID','timestamp','supply(kg)', 'price(원/kg)     │
│   2 train_df['item_id'] = train_df.ID.str[0:6]                                                   │
│   3                                                                                              │
│ ❱ 4 predictions_classfication_train_result = predict_model(llar, data = train_x_classificati     │
│   5                                                                                              │
│                                                                                                  │
│ /Users/shimjunseok/Library/Python/3.9/lib/python/site-packages/pycaret/classification/functional │
│ .py:2166 in predict_model                                                                        │
│                                                                                                  │
│   2163 │   if experiment is None:                                                                │
│   2164 │   │   experiment = _EXPERIMENT_CLASS()                                                  │
│   2165 │                                                                                         │
│ ❱ 2166 │   return experiment.predict_model(                                                      │
│   2167 │   │   estimator=estimator,                                                              │
│   2168 │   │   data=data,                                                                        │
│   2169 │   │   probability_threshold=probability_threshold,                                      │
│                                                                                                  │
│ /Users/shimjunseok/Library/Python/3.9/lib/python/site-packages/pycaret/classification/oop.py:281 │
│ 3 in predict_model                                                                               │
│                                                                                                  │
│   2810 │   │                                                                                     │
│   2811 │   │   """                                                                               │
│   2812 │   │                                                                                     │
│ ❱ 2813 │   │   return super().predict_model(                                                     │
│   2814 │   │   │   estimator=estimator,                                                          │
│   2815 │   │   │   data=data,                               

In [ ]:
train_df

In [14]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

# train_df = pd.read_csv('./data/train.csv')[['ID','timestamp','supply(kg)', 'price(원/kg)']]
# train_df['item_id'] = train_df.ID.str[0:6]
data = TimeSeriesDataFrame(train_df.drop(columns=['ID']))
predictor = TimeSeriesPredictor( 
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",
)

# seed 고정
predictor.fit( data, random_seed=42, )
predictor.refit_full()


pred = predictor.predict(data, random_seed=42, )


# submission = pd.read_csv('./sample_submission.csv')
# submission['answer'] = pred.reset_index()['mean']
# submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
# submission.to_csv('./dacon_submission.csv', index=False)
# submission

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231113_125906/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price(원/kg)'
	past covariates:  ['supply(kg)']

Starting training. Start time is 202

In [15]:
submission = pd.read_csv('./data/sample_submission.csv')
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
submission.to_csv('./submission/with_class_col_one_hot_encoder_submission.csv', index=False)
submission

,ID,answer
0,TG_A_J_20230304,3255.487589
1,TG_A_J_20230305,753.103822
2,TG_A_J_20230306,3043.293467
3,TG_A_J_20230307,3463.071807
4,TG_A_J_20230308,3327.123115
...,...,...
1087,RD_F_J_20230327,480.927193
1088,RD_F_J_20230328,496.371479
1089,RD_F_J_20230329,499.119709
1090,RD_F_J_20230330,483.295171


In [16]:
result = pd.DataFrame()
result['classification'] = predictions_classfication['prediction_label']
result['regression'] = submission['answer']
result.head()

,classification,regression
0,0,3255.487589
1,0,753.103822
2,1,3043.293467
3,1,3463.071807
4,1,3327.123115


In [17]:
for i in range(len(result)):
    if (i-1) % 7 == 0: # 1, 8, 15, 22
        result.iloc[i,-1] = 0

In [18]:
result['regression']

0        3255.487589
1           0.000000
2        3043.293467
3        3463.071807
4        3327.123115
            ...     
59392            NaN
59393            NaN
59394            NaN
59395            NaN
59396       0.000000
Name: regression, Length: 59397, dtype: float64

In [19]:
result1 = pd.DataFrame()
result1['ID'] = test['ID']
result1['answer'] = result['regression']

result1.to_csv('./submission/result_one_hot_encoder_submission_1113.csv', index=False)

In [20]:
result1

,ID,answer
0,TG_A_J_20230304,3255.487589
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3043.293467
3,TG_A_J_20230307,3463.071807
4,TG_A_J_20230308,3327.123115
...,...,...
1087,RD_F_J_20230327,480.927193
1088,RD_F_J_20230328,496.371479
1089,RD_F_J_20230329,499.119709
1090,RD_F_J_20230330,483.295171
